In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import tensorflow_datasets as tfds

#### Set fixed parameters

In [ ]:
IMG_SIZE = (299, 299)
NUM_CLASSES = 10
BATCH_SIZE = 32
EPOCHS = 10